<div class="alert alert-block alert-success">
    <h1 align="center">Robotics project</h1>
    <h3 align="center">Analyzing 6-DOF UR10 robot arm</h3>
    <h5 align="center">Phase2 (Inverse velocity and Dynamics)</h5>
</div>

## Import libraries

In [1]:
from sympy import *
import numpy as np
import math
from math import degrees

## Forward Kinematics

In [2]:
# Define the symbolic variables
num_symbols = 6  # Number of symbols to generate
alpha_names = [f"alpha{i}" for i in range(num_symbols+1)]
a_names = [f"a{i}" for i in range(num_symbols+1)]
d_names = [f"d{i}" for i in range(num_symbols+1)]
theta_names = [f"theta{i}" for i in range(num_symbols+1)]

alpha = symbols(alpha_names)
a = symbols(a_names)
d = symbols(d_names)
theta = symbols(theta_names)

# Define DH table
DH_param = [[0, 0, d[1], theta[1]],
            [pi/2, 0, 0, theta[2]],
            [0, a[2], 0, theta[3]],
            [0, a[3], d[4], theta[4]],
            [-pi/2, 0, d[5], theta[5]],
            [pi/2, 0, d[6], theta[6]]]

# find homogeneous transformations(T_i_i-1)                
T = []
for i in range(6):
    T_temp = [[cos(DH_param[i][3]), -sin(DH_param[i][3]), 0, DH_param[i][1]],
              [sin(DH_param[i][3])*cos(DH_param[i][0]),  cos(DH_param[i][3])*cos(DH_param[i][0]), -sin(DH_param[i][0]), -sin(DH_param[i][0])*(DH_param[i][2])],
              [sin(DH_param[i][3])*sin(DH_param[i][0]), cos(DH_param[i][3])*sin(DH_param[i][0]), cos(DH_param[i][0]), cos(DH_param[i][0])*DH_param[i][2]],
              [0, 0, 0, 1]]
    T.append(T_temp)

## Part1
### Finding Jacobian Matrix

#### 1-1) Find T_0_i (like T_0_1, T_0_2, ...)

In [3]:
T_0_i = [T[0]]
result = T[0]
for n in range(1,len(T)):
    # Multiply the matrices
    result = [[sum(result[i][k] * T[n][k][j] for k in range(4)) for j in range(4)] for i in range(4)]
    # Simplify the result according to cosine multiplication rules
    simplified_result = [[trigsimp(expr) for expr in row] for row in result]
    T_0_i.append(simplified_result)

#### 1-2) Find zi and oi

In [4]:
z = []
o = []
for T0i in T_0_i:
    z.append(np.array(T0i)[:3,2][np.newaxis].T)
    o.append(np.array(T0i)[:3,3][np.newaxis].T)

#### 1-3) Creat J(theta)

Here all joints are Revolute. So we use below formula for creating J(theta): 
 
    Ji = [[zi*(oe-oi)],[zi]]

In [5]:
J = []
for i in range(len(z)):
    Jvi = np.cross(z[i].T, (o[len(o)-1]-o[i]).T).T
    Jwi = z[i]
    Ji = np.append(Jvi,Jwi)
    J.append(Ji)
J = np.array([[trigsimp(expr) for expr in row] for row in J]).T

In [6]:
Matrix(J)

Matrix([
[-a2*sin(theta1)*cos(theta2) - a3*sin(theta1)*cos(theta2 + theta3) + d4*cos(theta1) + d5*sin(theta1)*sin(theta2 + theta3 + theta4) + d6*(-sin(theta1)*sin(theta5)*cos(theta2 + theta3 + theta4) + cos(theta1)*cos(theta5)), -(a2*sin(theta2) + a3*sin(theta2 + theta3) + d5*cos(theta2 + theta3 + theta4) + d6*sin(theta5)*sin(theta2 + theta3 + theta4))*cos(theta1), -(a3*sin(theta2 + theta3) + d5*cos(theta2 + theta3 + theta4) + d6*sin(theta5)*sin(theta2 + theta3 + theta4))*cos(theta1), -(d5*cos(theta2 + theta3 + theta4) + d6*sin(theta5)*sin(theta2 + theta3 + theta4))*cos(theta1), d6*(-sin(theta1)*sin(theta5) + cos(theta1)*cos(theta5)*cos(theta2 + theta3 + theta4)),                                                                               0],
[  a2*cos(theta1)*cos(theta2) + a3*cos(theta1)*cos(theta2 + theta3) + d4*sin(theta1) - d5*sin(theta2 + theta3 + theta4)*cos(theta1) + d6*(sin(theta1)*cos(theta5) + sin(theta5)*cos(theta1)*cos(theta2 + theta3 + theta4)), -(a2*sin(theta2) + a3*sin

## Part2
### Find Singularities

### 2-1) J singularities

In [196]:
J_det = Matrix(J).det()

In [197]:
J_det_simp = trigsimp(J_det)

In [198]:
J_det_simp

a2*a3*(a2*cos(theta2) + a3*cos(theta2 + theta3) - d5*sin(theta2 + theta3 + theta4))*sin(theta3)*sin(theta5)

### 2-2) Jv singularities

In [ ]:
Jv = J[:3,:]
simple_JvJvT = simplify(Jv@(Jv.T))

In [ ]:
Jv_det = Matrix(Jv@Jv.T).det()
Jv_det_simp = trigsimp(Jv_det)
Jv_det_simp

### 2-3) Jw singularities

In [ ]:
Jw = J[3:,:]
simple_JwJwT = simplify(Jw@(Jw.T))

In [ ]:
Jw_det = Matrix(simple_JwJwT).det()
Jw_det_simp = trigsimp(Jw_det)
Jw_det_simp

## Part3
### Analisys of singularities

## Part4
### Dynamics Equations (Newton-ٍEuler method)

#### 4-1) Define suitable variables 

- We have Rotation matrixes from forward kinematics. Here we need both R and R transpose:

In [8]:
DOF = 6
# Some variables which we had them from Phase1 and robot structure
R = [np.array(T[i])[:3,:3] for i in range(DOF)]
R_trans = [np.array(T[i])[:3,:3].T for i in range(DOF)]

- We need Inertia tensors of the links. We find them from Solidworks file.

In [9]:
# Units : kg.m^2
I_c0_0 = np.array([[0, 0, 0],[0, 0, 0],[0, 0, 0]])
I_c1_1 = np.array([[0.03, 0, 0],[0, 0.03, 0],[0, 0, 0.03]])
I_c2_2 = np.array([[0.05, 0, 0.01],[0, 1.23, 0],[0.01, 0, 1.23]])
I_c3_3 = np.array([[0.02, 0, 0],[0, 0.54, 0],[0, 0, 0.54]])
I_c4_4 = np.array([[0.003, 0, 0],[0, 0.0024, 0.00025],[0, 0.00025, 0.0028]])
I_c5_5 = np.array([[0.003, 0, 0],[0, 0.0024, -0.00025],[0, -0.00025, 0.0028]])
I_c6_6 = np.array([[0.00022, 0, 0],[0, 0.00024, 0],[0, 0, 0.0004]])
I = [I_c0_0, I_c1_1, I_c2_2, I_c3_3, I_c4_4, I_c5_5, I_c6_6]

- We need mass of each link. We find them from Solidworks file.

In [11]:
# Units : kg
m0, m1, m2, m3, m4, m5, m6 = 0, 8.3, 23.52, 12.56, 1.967, 1.967, 0.43

- We need the distance of center of mass of link i from frame i. (P_i_ci)

In [15]:
P_0_c0 = np.array([[0],[0],[0]])
P_1_c1 = np.array([[0],[-0.01],[-0.01]])*10**(-3)
P_2_c2 = np.array([[0.25],[0],[0.17]])*10**(-3)
P_3_c3 = np.array([[0.26],[0],[0.05]])*10**(-3)
P_4_c4 = np.array([[0],[9.74],[-7.6]])*10**(-3)
P_5_c5 = np.array([[0],[-9.74],[-7.6]])*10**(-3)
P_6_c6 = np.array([[0],[-0.95],[-17.46]])*10**(-3)
Pc = [P_0_c0, P_1_c1, P_2_c2, P_3_c3, P_4_c4, P_5_c5, P_6_c6]

- We need the distance between frame i+1 and i. We can find them from fourth column of forward kinematics transformation matrix

In [22]:
T_num = T
T_num = [Matrix(T_num[i]).subs({'d1':10, 'd4': 20, 'd5':8, 'd6':8, 'a2':9, 'a3':10}) for i in range(len(T_num))]

NameError: name 'd4' is not defined

In [20]:
Matrix(T_num[1])

Matrix([
[cos(theta2), -sin(theta2),  0, 0],
[          0,            0, -1, 0],
[sin(theta2),  cos(theta2),  0, 0],
[          0,            0,  0, 1]])

In [41]:
DOF = 6
# Some variables which we had them from Phase1 and robot structure
R = [np.array(T[i])[:3,:3] for i in range(DOF)]
R_trans = [np.array(T[i])[:3,:3].T for i in range(DOF)]
I = [0,]
P = []
Pc = [0,]
m = [0,]
# some parametric varibales
num_symbols = 6  # Number of symbols to generate
theta_d_names = [f"theta_d{i}" for i in range(num_symbols+1)]
theta_d = symbols(theta_d_names)
theta_dd_names = [f"theta_dd{i}" for i in range(num_symbols+1)]
theta_dd = symbols(theta_dd_names)
# Some varibales we should expand them in Outward iterations
w = [np.array([[0],[0],[0]])]
w_dot = [np.array([[0],[0],[0]])]
v_dot = [np.array([[0],[0],[0]])]
vc_dot = []
F = [np.array([[0],[0],[0]])]
N = [np.array([[0],[0],[0]])]
# Some varibales we should expand them in Inward iterations
f = [np.array([[0],[0],[0]])]
n = [np.array([[0],[0],[0]])]
taw = []

#### 4-2) Outward iterations

In [ ]:
for i in range(DOF):
    
    w_i1 = R_trans[i]@w[i] + np.array([[0],[0],[theta_d[i+1]]])
    w_dot_i1 = R_trans[i]@w_dot[i] + np.cross(R_trans[i]@w[i], np.array([[0],[0],[theta_d[i+1]]])) + np.array([[0],[0],[theta_dd[i+1]]])
    v_dot_i1 = R_trans[i]@(np.cross(w_dot[i], P[i])+np.cross(w[i],np.cross(w[i],P[i]))+v_dot[i])
    vc_dot_i1 = np.cross(w_dot_i1, Pc[i+1])+np.cross(w_i1,np.cross(w_i1,Pc[i+1]))+v_dot_i1
    F_i1 = m[i+1]*vc_dot_i1
    N_i1 = I[i+1]@w_dot_i1 + np.cross(w_i1, I[i+1]@w_i1)
    
    w.append(w_i1)
    w_dot.append(w_dot_i1)
    v_dot.append(v_dot_i1)
    vc_dot.append(vc_dot_i1)
    F.append(F_i1)
    N.append(N_i1)

#### 4-3) Inward iterations

In [ ]:
for i in range(DOF,0,-1):
    fi = R[i]@f[len(f)-1] + F[i]
    ni = N[i] + R[i]@n[len(n)-1]+np.cross(Pc[i],F[i])+np.cross(P[i], R[i]@f[len(f)-1])
    tawi = ni[2]
    f.append(fi)
    n.append(ni)
    taw.append(tawi)
f.reverse()
n.reverse()
taw.reverse()

In [98]:
F = np.empty((3, 0), float)

In [100]:
np.append(F,np.array([[[1],[1],[1]]]), axis =0)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 3 dimension(s)

In [114]:
v3 = [np.array([[0],[0],[0]])]

In [115]:
v3.append(np.array([[1],[1],[2]]))

In [116]:
v3

[array([[0],
        [0],
        [0]]),
 array([[1],
        [1],
        [2]])]

In [13]:
10**(-3)*np.array([[0.26],[0],[0.05]])

array([[2.6e-04],
       [0.0e+00],
       [5.0e-05]])